In [10]:
import pandas as pd
no_panel = pd.read_csv("../csdid_comparing/data/R_nopanel_reg.csv")
y = no_panel['y']
post = no_panel['post']
d = no_panel['d']
x = None
w = no_panel['i_w']

In [11]:
import numpy as np
import statsmodels.api as sm

def drdid_rc(y, post, D, covariates, i_weights=None):
    # Convert inputs to numpy arrays
    D = np.asarray(D).flatten()
    n = len(D)
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()
    
    # Add constant to covariate matrix
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    # Normalize weights
    if i_weights is None:
        i_weights = np.ones(n)
    i_weights = np.asanyarray(i_weights).flatten()
    i_weights = i_weights / np.mean(i_weights)
    
    # Compute the Pscore by MLE
    pscore_model = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights)
    pscore_results = pscore_model.fit()
    if not pscore_results.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(pscore_results.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = pscore_results.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)
    
    # Compute the Outcome regression for the control group at the pre-treatment period
    mask_cont_pre = (D == 0) & (post == 0)
    reg_cont_pre = sm.WLS(y[mask_cont_pre], int_cov[mask_cont_pre], weights=i_weights[mask_cont_pre]).fit()
    if np.any(np.isnan(reg_cont_pre.params)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    out_y_cont_pre = np.dot(int_cov, reg_cont_pre.params)
    
    # Compute the Outcome regression for the control group at the post-treatment period
    mask_cont_post = (D == 0) & (post == 1)
    reg_cont_post = sm.WLS(y[mask_cont_post], int_cov[mask_cont_post], weights=i_weights[mask_cont_post]).fit()
    if np.any(np.isnan(reg_cont_post.params)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    out_y_cont_post = np.dot(int_cov, reg_cont_post.params)
    
    # Combine the ORs for control group
    out_y_cont = post * out_y_cont_post + (1 - post) * out_y_cont_pre
    
    # Compute the Outcome regression for the treated group at the pre-treatment period
    mask_treat_pre = (D == 1) & (post == 0)
    reg_treat_pre = sm.WLS(y[mask_treat_pre], int_cov[mask_treat_pre], weights=i_weights[mask_treat_pre]).fit()
    out_y_treat_pre = np.dot(int_cov, reg_treat_pre.params)
    
    # Compute the Outcome regression for the treated group at the post-treatment period
    mask_treat_post = (D == 1) & (post == 1)
    reg_treat_post = sm.WLS(y[mask_treat_post], int_cov[mask_treat_post], weights=i_weights[mask_treat_post]).fit()
    out_y_treat_post = np.dot(int_cov, reg_treat_post.params)
    
    # Compute weights
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)
    w_d = i_weights * D
    w_dt1 = i_weights * D * post
    w_dt0 = i_weights * D * (1 - post)
    
    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * (y - out_y_cont) / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * (y - out_y_cont) / np.mean(w_treat_post)
    eta_cont_pre = w_cont_pre * (y - out_y_cont) / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * (y - out_y_cont) / np.mean(w_cont_post)
    
    # Extra elements for the locally efficient DRDID
    eta_d_post = w_d * (out_y_treat_post - out_y_cont_post) / np.mean(w_d)
    eta_dt1_post = w_dt1 * (out_y_treat_post - out_y_cont_post) / np.mean(w_dt1)
    eta_d_pre = w_d * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_d)
    eta_dt0_pre = w_dt0 * (out_y_treat_pre - out_y_cont_pre) / np.mean(w_dt0)
    
    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)
    att_d_post = np.mean(eta_d_post)
    att_dt1_post = np.mean(eta_dt1_post)
    att_d_pre = np.mean(eta_d_pre)
    att_dt0_pre = np.mean(eta_dt0_pre)
    
    # ATT estimator
    dr_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre) + \
             (att_d_post - att_dt1_post) - (att_d_pre - att_dt0_pre)
    
    # Compute influence functions
    weights_ols_pre = i_weights * (1 - D) * (1 - post)
    weights_ols_pre = np.array(weights_ols_pre)
    wols_x_pre = weights_ols_pre[:, np.newaxis] * int_cov
    wols_eX_pre = weights_ols_pre[:, np.newaxis] * (y - out_y_cont_pre)[:, np.newaxis] * int_cov
    XpX_inv_pre = np.linalg.inv(np.dot(wols_x_pre.T, int_cov) / n)
    asy_lin_rep_ols_pre = np.dot(wols_eX_pre, XpX_inv_pre)
    
    weights_ols_post = i_weights * (1 - D) * post
    
    wols_x_post = weights_ols_post[:, np.newaxis] * int_cov
    wols_eX_post = weights_ols_post[:, np.newaxis] * (y - out_y_cont_post)[:, np.newaxis] * int_cov
    XpX_inv_post = np.linalg.inv(np.dot(wols_x_post.T, int_cov) / n)
    asy_lin_rep_ols_post = np.dot(wols_eX_post, XpX_inv_post)
    
    weights_ols_pre_treat = i_weights * D * (1 - post)
    wols_x_pre_treat = weights_ols_pre_treat[:, np.newaxis] * int_cov
    wols_eX_pre_treat = weights_ols_pre_treat[:, np.newaxis] * (y - out_y_treat_pre)[:, np.newaxis] * int_cov
    XpX_inv_pre_treat = np.linalg.inv(np.dot(wols_x_pre_treat.T, int_cov) / n)
    asy_lin_rep_ols_pre_treat = np.dot(wols_eX_pre_treat, XpX_inv_pre_treat)
    
    weights_ols_post_treat = i_weights * D * post
    wols_x_post_treat = weights_ols_post_treat[:, np.newaxis] * int_cov
    wols_eX_post_treat = weights_ols_post_treat[:, np.newaxis] * (y - out_y_treat_post)[:, np.newaxis] * int_cov
    XpX_inv_post_treat = np.linalg.inv(np.dot(wols_x_post_treat.T, int_cov) / n)
    asy_lin_rep_ols_post_treat = np.dot(wols_eX_post_treat, XpX_inv_post_treat)
    
    score_ps = i_weights[:, np.newaxis] * (D - ps_fit)[:, np.newaxis] * int_cov
    Hessian_ps = pscore_results.cov_params() * n
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)
    
    # Influence function components
    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre / np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post / np.mean(w_treat_post)
    
    M1_post = -np.mean(w_treat_post[:, np.newaxis] * post[:, np.newaxis] * int_cov, axis=0) / np.mean(w_treat_post)
    M1_pre = -np.mean(w_treat_pre[:, np.newaxis] * (1 - post)[:, np.newaxis] * int_cov, axis=0) / np.mean(w_treat_pre)
    
    inf_treat_or_post = np.dot(asy_lin_rep_ols_post, M1_post)
    inf_treat_or_pre = np.dot(asy_lin_rep_ols_pre, M1_pre)
    inf_treat_or = inf_treat_or_post + inf_treat_or_pre
    
    inf_treat = inf_treat_post - inf_treat_pre + inf_treat_or
    
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre / np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post / np.mean(w_cont_post)
    
    M2_pre = np.mean(w_cont_pre[:, np.newaxis] * (y - out_y_cont - att_cont_pre)[:, np.newaxis] * int_cov, axis=0) / np.mean(w_cont_pre)
    M2_post = np.mean(w_cont_post[:, np.newaxis] * (y - out_y_cont - att_cont_post)[:, np.newaxis] * int_cov, axis=0) / np.mean(w_cont_post)
    
    inf_cont_ps = np.dot(asy_lin_rep_ps, M2_post - M2_pre)
    
    M3_post = -np.mean(w_cont_post[:, np.newaxis] * post[:, np.newaxis] * int_cov, axis=0) / np.mean(w_cont_post)
    M3_pre = -np.mean(w_cont_pre[:, np.newaxis] * (1 - post)[:, np.newaxis] * int_cov, axis=0) / np.mean(w_cont_pre)
    
    inf_cont_or_post = np.dot(asy_lin_rep_ols_post, M3_post)
    inf_cont_or_pre = np.dot(asy_lin_rep_ols_pre, M3_pre)
    inf_cont_or = inf_cont_or_post + inf_cont_or_pre
    
    inf_cont = inf_cont_post - inf_cont_pre + inf_cont_ps + inf_cont_or
    
    dr_att_inf_func1 = inf_treat - inf_cont
    
    inf_eff1 = eta_d_post - w_d * att_d_post / np.mean(w_d)
    inf_eff2 = eta_dt1_post - w_dt1 * att_dt1_post / np.mean(w_dt1)
    inf_eff3 = eta_d_pre - w_d * att_d_pre / np.mean(w_d)
    inf_eff4 = eta_dt0_pre - w_dt0 * att_dt0_pre / np.mean(w_dt0)
    inf_eff = (inf_eff1 - inf_eff2) - (inf_eff3 - inf_eff4)
    
    mom_post = np.mean((w_d[:, np.newaxis] / np.mean(w_d) - w_dt1[:, np.newaxis] / np.mean(w_dt1)) * int_cov, axis=0)
    mom_pre = np.mean((w_d[:, np.newaxis] / np.mean(w_d) - w_dt0[:, np.newaxis] / np.mean(w_dt0)) * int_cov, axis=0)
    inf_or_post = np.dot(asy_lin_rep_ols_post_treat - asy_lin_rep_ols_post, mom_post)
    inf_or_pre = np.dot(asy_lin_rep_ols_pre_treat - asy_lin_rep_ols_pre, mom_pre)
    inf_or = inf_or_post - inf_or_pre
    
    dr_att_inf_func = dr_att_inf_func1 + inf_eff + inf_or
    
    return dr_att, dr_att_inf_func

drdid_rc(y, post, d, x, w)

(-32.44762929840418,
 array([ 4.20279333e+02,  4.12088741e+02, -9.76295144e+01,  4.13877211e+02,
        -1.33180878e+02,  3.46372794e+02,  3.33751840e+01, -2.36632190e+02,
        -2.87278416e+02, -5.01885265e+01, -2.56919638e+01, -4.36605269e+02,
        -5.82711118e+02, -1.74750149e+02, -1.05917795e+02,  3.17576284e+02,
         5.22646939e+02, -3.92097558e+02, -1.74362318e+02,  3.25937355e+02,
         1.15121568e+01,  9.67115269e+01,  2.37900939e+02, -2.31235308e+02,
        -1.67406181e+02,  2.49385835e+02,  7.23160188e+02,  1.39005288e+02,
        -5.82824544e+01, -2.09472417e+02, -1.67529498e+02, -1.98673017e+02,
        -1.10899807e+02, -2.23694179e+01, -8.20674968e+02,  5.35943555e+02,
         6.10833988e+02,  4.23615577e+02,  1.39299854e+02,  2.29206492e+01,
        -6.33676385e+02, -3.95707388e+02,  1.89663298e+01,  1.01334683e+02,
         6.97094075e+01, -1.95271048e+02, -3.44512247e+01, -6.72134538e+02,
        -2.95689468e+02,  2.80919019e+01, -2.30572474e+02,  3.06118

In [50]:
def std_ipw_did_rc(y, post, D, covariates, i_weights = None):
    D = np.asarray(D).flatten()
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()
    n = len(D)
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    # Pesos
    if i_weights is None:
        i_weights = np.ones(n)
    else:
        i_weights = np.asarray(i_weights)
        if np.min(i_weights) < 0:
            raise ValueError("i_weights must be non-negative")
    
    # Normalizar pesos
    i_weights = np.asarray(i_weights).flatten()
    i_weights = i_weights / np.mean(i_weights)

    pscore_model = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights)
    pscore_results = pscore_model.fit()
    if not pscore_results.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(pscore_results.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = pscore_results.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)


    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    # print(np.mean(w_treat_pre))

    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post)/(1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post/(1 - ps_fit)

    # Elements of the influence function (summands)
    eta_treat_pre = w_treat_pre * y / np.mean(w_treat_pre)
    eta_treat_post = w_treat_post * y / np.mean(w_treat_post)
    # print(eta_treat_pre)

    eta_cont_pre = w_cont_pre * y / np.mean(w_cont_pre)
    eta_cont_post = w_cont_post * y / np.mean(w_cont_post)

    # Estimator of each component
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)

    score_ps = (i_weights * (D - ps_fit))[:, np.newaxis] * int_cov
    Hessian_ps = pscore_results.cov_params() * n
    asy_lyn_rep_ps = np.dot(score_ps, Hessian_ps)

    inf_treat_pre = eta_treat_pre - w_treat_pre * att_treat_pre/np.mean(w_treat_pre)
    inf_treat_post = eta_treat_post - w_treat_post * att_treat_post/np.mean(w_treat_post)
    inf_treat = inf_treat_post - inf_treat_pre
    # Now, get the influence function of control component
    # Leading term of the influence function: no estimation effect
    inf_cont_pre = eta_cont_pre - w_cont_pre * att_cont_pre/np.mean(w_cont_pre)
    inf_cont_post = eta_cont_post - w_cont_post * att_cont_post/np.mean(w_cont_post)
    inf_cont = inf_cont_post - inf_cont_pre

    # Estimation effect from gamma hat (pscore)
    # Derivative matrix (k x 1 vector)
  
    M2_pre = np.mean((w_cont_pre *(y - att_cont_pre))[:, np.newaxis] * int_cov, axis = 0)/np.mean(w_cont_pre)
    M2_post = np.mean((w_cont_post *(y - att_cont_post))[:, np.newaxis] * int_cov, axis = 0)/np.mean(w_cont_post)

    # Now the influence function related to estimation effect of pscores
    M2 = M2_post - M2_pre
    # print()

    inf_cont_ps = np.dot(asy_lyn_rep_ps, M2)

    # Influence function for the control component
    inf_cont = inf_cont + inf_cont_ps

    #get the influence function of the DR estimator (put all pieces together)
    att_inf_func = inf_treat - inf_cont
    # print(np.std(att_inf_func) / np.sqrt(n))
    return ipw_att, att_inf_func

std_ipw_did_rc(y, post, d, x, w)

11.18310945627825


(-32.44762929840434,
 array([ 4.20279333e+02,  4.12088741e+02, -9.76295144e+01,  4.13877211e+02,
        -1.33180878e+02,  3.46372794e+02,  3.33751840e+01, -2.36632190e+02,
        -2.87278416e+02, -5.01885265e+01, -2.56919638e+01, -4.36605269e+02,
        -5.82711118e+02, -1.74750149e+02, -1.05917795e+02,  3.17576284e+02,
         5.22646939e+02, -3.92097558e+02, -1.74362318e+02,  3.25937355e+02,
         1.15121568e+01,  9.67115269e+01,  2.37900939e+02, -2.31235308e+02,
        -1.67406181e+02,  2.49385835e+02,  7.23160188e+02,  1.39005288e+02,
        -5.82824544e+01, -2.09472417e+02, -1.67529498e+02, -1.98673017e+02,
        -1.10899807e+02, -2.23694179e+01, -8.20674968e+02,  5.35943555e+02,
         6.10833988e+02,  4.23615577e+02,  1.39299854e+02,  2.29206492e+01,
        -6.33676385e+02, -3.95707388e+02,  1.89663298e+01,  1.01334683e+02,
         6.97094075e+01, -1.95271048e+02, -3.44512247e+01, -6.72134538e+02,
        -2.95689468e+02,  2.80919019e+01, -2.30572474e+02,  3.06118

In [16]:
import numpy as np
import statsmodels.api as sm

def ipw_did_rc(y, post, D, covariates=None, i_weights=None):
    # D, y, post como arrays numpy
    D = np.asarray(D).flatten()
    y = np.asarray(y).flatten()
    post = np.asarray(post).flatten()
    n = len(D)
    
    # Agregar constante a covariates si existe
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    # Pesos
    if i_weights is None:
        i_weights = np.ones(n)
    else:
        i_weights = np.asarray(i_weights)
        if np.min(i_weights) < 0:
            raise ValueError("i_weights must be non-negative")
    
    # Normalizar pesos
    i_weights = np.asarray(i_weights).flatten()
    i_weights = i_weights / np.mean(i_weights)
    
    # Estimación de Propensity Score (logit)
    pscore_model = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights)
    pscore_results = pscore_model.fit()
    if not pscore_results.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(pscore_results.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = pscore_results.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)
    
    # Calcular IPW estimator
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont_pre = i_weights * ps_fit * (1 - D) * (1 - post) / (1 - ps_fit)
    w_cont_post = i_weights * ps_fit * (1 - D) * post / (1 - ps_fit)
    
    Pi_hat = np.mean(i_weights * D)
    lambda_hat = np.mean(i_weights * post)
    one_minus_lambda_hat = np.mean(i_weights * (1 - post))
    
    eta_treat_pre = w_treat_pre * y / (Pi_hat * one_minus_lambda_hat)
    eta_treat_post = w_treat_post * y / (Pi_hat * lambda_hat)
    eta_cont_pre = w_cont_pre * y / (Pi_hat * one_minus_lambda_hat)
    eta_cont_post = w_cont_post * y / (Pi_hat * lambda_hat)
    
    att_treat_pre = np.mean(eta_treat_pre)
    att_treat_post = np.mean(eta_treat_post)
    att_cont_pre = np.mean(eta_cont_pre)
    att_cont_post = np.mean(eta_cont_post)
    
    ipw_att = (att_treat_post - att_treat_pre) - (att_cont_post - att_cont_pre)
    
    # Cálculo de la función de influencia
    score_ps = i_weights[:, np.newaxis] * (D - ps_fit)[:, np.newaxis] * int_cov
    Hessian_ps = pscore_results.cov_params() * n
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)
    
    inf_treat_post = (eta_treat_post - att_treat_post) - \
                     (i_weights * D - Pi_hat) * att_treat_post / Pi_hat - \
                     (i_weights * post - lambda_hat) * att_treat_post / lambda_hat
    
    inf_treat_pre = (eta_treat_pre - att_treat_pre) - \
                    (i_weights * D - Pi_hat) * att_treat_pre / Pi_hat - \
                    (i_weights * (1 - post) - one_minus_lambda_hat) * att_treat_pre / one_minus_lambda_hat
    
    inf_cont_post = (eta_cont_post - att_cont_post) - \
                    (i_weights * D - Pi_hat) * att_cont_post / Pi_hat - \
                    (i_weights * post - lambda_hat) * att_cont_post / lambda_hat
    
    inf_cont_pre = (eta_cont_pre - att_cont_pre) - \
                   (i_weights * D - Pi_hat) * att_cont_pre / Pi_hat - \
                   (i_weights * (1 - post) - one_minus_lambda_hat) * att_cont_pre / one_minus_lambda_hat
    
    mom_logit_pre = -eta_cont_pre[:, np.newaxis] * int_cov
    mom_logit_pre = np.mean(mom_logit_pre, axis=0)
    
    mom_logit_post = -eta_cont_post[:, np.newaxis] * int_cov
    mom_logit_post = np.mean(mom_logit_post, axis=0)
    
    inf_logit = asy_lin_rep_ps @ (mom_logit_post - mom_logit_pre)
    
    att_inf_func = (inf_treat_post - inf_treat_pre) - (inf_cont_post - inf_cont_pre) + inf_logit
    print(np.std(att_inf_func) / np.sqrt(n))
    return ipw_att, att_inf_func
ipw_did_rc(y, post, d, x, w)

58.43262881229079


(-93.44964719410004,
 array([-1771.69299088,  2734.43642941,  -665.22558666, -1301.21607242,
         -996.11260525, -2158.69871885,  -504.89793626,  2423.86068757,
         1877.09030231,  -719.10776276,  1235.01984081, -3215.35303448,
         2386.43646889, -2175.74096025, -1086.57851202,  2126.26135247,
        -1623.59474401,  1273.22968698,   894.95899003,  1285.88717946,
         2013.14752053, -1315.44015778, -1026.47833569, -2502.18129446,
         1564.16066321, -1200.80571295,  3687.54887082, -2585.69577723,
         1737.14743007,  2156.46916393, -1131.66288093,  1072.00424587,
          356.97367802,   351.36368266,  1595.18126677, -1323.98704545,
        -1255.93969133, -1506.35891117, -1584.95043603,   479.97949402,
        -3282.15157163,   710.78455687, -1417.11138741,  2501.57210618,
          961.07317252, -1683.69307958,   296.03996423,   879.46613051,
         2340.35863086, -1611.71256146,  1097.07919423, -1940.81707908,
          480.70920993, -2786.26114977, -11

In [9]:
w

0      0.683004
1      0.658181
2      0.151920
3      0.528905
4      0.246862
         ...   
995    0.127359
996    0.606165
997    0.856390
998    0.333366
999    0.438967
Name: i_w, Length: 1000, dtype: float64